In [ ]:
import Quant.Value as qv
import numpy as np
import pandas as pd
from sklearn.decomposition import SparsePCA
import matplotlib.pyplot as plt

tickers, d = qv.get_all_tickers('./tickers.csv')

balance, price_data, info, raw_info = qv.Book_to_Market_Ratio_Multi_Asset(tickers)

Extensions to add: alpha_vantage@1.5.0, biztoc@1.5.0, cboe@1.5.0, deribit@1.1.0, ecb@1.5.0, econometrics@1.6.0, famafranch@1.1.0, famafrench@1.1.0, finra@1.5.0, finviz@1.4.0, government_us@1.5.0, multpl@1.2.0, nasdaq@1.5.0, openbb_charting@2.4.0, quantitative@1.5.0, seeking_alpha@1.5.0, stockgrid@1.5.0, technical@1.5.0, tmx@1.4.0, tradier@1.4.0, wsj@1.5.0

Building...


In [ ]:
info = info.apply(pd.to_numeric, errors='ignore')
X = info.select_dtypes(include='number')
X = X.drop(columns=['recommendationMean'])
print(f"Found {X.shape[1]} numerical features")
X = X.dropna(axis=1, thresh=int(0.8 * len(X)))
X_filled_mean = X.fillna(X.mean())
X_filled_zeros = X.fillna(0)
# X_winsorized = X.apply(lambda x: winsorize(x, limits=[0.06, 0.06]))
x_mean = X.mean()
x_std = X.std()
X_normalized = (X_filled_mean - x_mean) / x_std
X_normalized = X_normalized.values






spca = SparsePCA(n_components=18, alpha=5, random_state=42)
Z_spca = spca.fit_transform(X_normalized)
raw_components = spca.components_.T
components = raw_components / x_std.values[:, np.newaxis]
components = components / np.linalg.norm(components, axis=0)
components = components.round(5)
Z_approx = Z_spca @ components.T
spca_numbers_filled_mean = pd.DataFrame(X_filled_zeros @ components).fillna((X_filled_mean @ components).mean()).values
info_spca = pd.DataFrame(X_filled_zeros @ components, index=info.index)
info_spca.columns = [f'SPC{i+1}' for i in range(info_spca.shape[1])]
# info_spca.columns = ['Liquidity','Profitability','Profit Efficiency','Revenue','Value']
info_spca = pd.concat([info.select_dtypes(exclude='number'), info['recommendationMean'], info_spca], axis=1)



A = X_normalized.T @ X_normalized
eig_values = [np.linalg.norm(A @ raw_components[:, i]).round() for i in range(raw_components.shape[1])]
# sort components by eigenvalues
mask = np.argsort(eig_values)[::-1]
components = raw_components[:, mask]

components = pd.DataFrame(components, index=X.columns)

mask = info['recommendationMean'].notna()
corr = [np.corrcoef(info['recommendationMean'][mask], spca_numbers_filled_mean[mask, i])[0,1] for i in range(spca_numbers_filled_mean.shape[1])]

corr_str = ', '.join([f'{x_corr:.4f}' for x_corr in corr])
print(f"Correlations between SPCs and recommendationMean: [{corr_str}]")
mask = np.argsort(np.abs(corr))[::-1]
print("SPC indices sorted by absolute correlation with recommendationMean:")
print(mask)

eig_values_str = ', '.join([f'{val}' for val in eig_values])
print(f"Eigenvalues of SPCs: [{eig_values_str}]")

OpenBBError: 
[Error] -> 1 validations error(s)
[Arg] limit -> input: 100 -> Input should be less than or equal to 5